# Deploy an MLflow model with SageMaker

## Install MLflow

In [ ]:
!pip install -q mlflow==2.6.0

## Setup environment

In [ ]:
import json
import boto3
import mlflow
import sagemaker
import pandas as pd
import mlflow.sagemaker
from sklearn.datasets import load_boston
from mlflow.deployments import get_deploy_client

# name of the AWS region to which to deploy the application
region = sagemaker.Session().boto_region_name
# we are using the notebook instance role for training in this example
role = sagemaker.get_execution_role() 
# uri of your remote mlflow server
tracking_uri = '<YOUR MLFLOW SERVER URI>' 
# set remote mlflow server
mlflow.set_tracking_uri(tracking_uri)

## Build MLflow docker image to serve the model with SageMaker 

In [ ]:
!mlflow sagemaker build-and-push-container

In [ ]:
# URL of the ECR-hosted Docker image the model should be deployed into
image_uri = '<YOUR mlflow-pyfunc ECR IMAGE URI>'

## Deploy a SageMaker endpoint with our scikit-learn model

In [ ]:
endpoint_name = 'boston-housing'
# The location, in URI format, of the MLflow model to deploy to SageMaker.
model_uri = 'models:/boston/1'

In [ ]:
config={
    'execution_role_arn': role,
    'image_url': image_uri,
    'instance_type': 'ml.m5.xlarge',
    'instance_count': 1, 
    'region_name': region
}

client = get_deploy_client("sagemaker")

client.create_deployment(
    name=endpoint_name,
    model_uri=model_uri,
    flavor='python_function',
    config=config
)

## Predict

In [ ]:
# load boston dataset
data = load_boston()
df = pd.DataFrame(data.data, columns=data.feature_names)

In [ ]:
client = get_deploy_client(f"sagemaker:/{region}")

payload = df.iloc[[0]]
prediction = client.predict(endpoint_name, df.iloc[[0]])

print(f'Payload: {payload}')
print(f'Prediction: {prediction}')

## Delete endpoint

In [ ]:
client.delete_deployment(endpoint_name, config=config)